In [1]:
# Load necessary libraries for ECG data processing
from pathlib import Path
import json
import numpy as np
import pandas as pd
import neurokit2 as nk
import matplotlib.pyplot as plt
import mne
%matplotlib qt

In [2]:

ecg_path  = "../../data/sub-032301(1)/sub-032301/ses-02/func/sub-032301_ses-02_task-rest_acq-AP_run-01_recording-ecg_physio.tsv.gz"
json_path = "../../data/sub-032301(1)/sub-032301/ses-02/func/sub-032301_ses-02_task-rest_acq-AP_run-01_recording-ecg_physio.json"

In [3]:
raw_ecg = pd.read_csv(ecg_path, sep='\t', compression='gzip')
raw_ecg = raw_ecg.iloc[:, :-1]

with open(json_path) as f:
    metadata = json.load(f)

sampling_rate = metadata["SamplingFrequency"]
start_time = metadata.get("StartTime")

columns = metadata.get("Columns")
if columns and len(columns) == raw_ecg.shape[1]:
    raw_ecg.columns = columns
print(raw_ecg.head())
cutoff = 919740  # just before the big artifacts
raw_ecg = raw_ecg.iloc[:cutoff].copy()


         ecg  trigger
0  171.62367      0.0
1  188.05879      0.0
2  205.13430      0.0
3  234.01009      0.0
4  275.68307      0.0


In [4]:
# Plot raw ECG signal
ecg_signal = raw_ecg["ecg"].to_numpy()
nk.signal_plot(ecg_signal, sampling_rate=sampling_rate, title="ECG (ses-02 PA run-01)")

In [5]:
# Plot an histogram of the ECG values
plt.figure(figsize=(8,4))
plt.hist(raw_ecg["ecg"], bins=100, alpha=0.7)
plt.xlabel("Amplitude")
plt.ylabel("Count")
plt.title("ECG histogram")
plt.show()


In [10]:
# Compute HRV metrics
ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=sampling_rate)
ecg_peaks, info = nk.ecg_peaks(ecg_cleaned, sampling_rate=sampling_rate)
hrv_metrics = nk.hrv(ecg_peaks, sampling_rate=sampling_rate, show=True)
print(hrv_metrics)

   HRV_MeanNN   HRV_SDNN  HRV_SDANN1  HRV_SDNNI1  HRV_SDANN2  HRV_SDNNI2  \
0  848.989834  56.343985    5.638586   50.173541    4.359563   54.772279   

   HRV_SDANN5  HRV_SDNNI5  HRV_RMSSD   HRV_SDSD  ...  HRV_SampEn  HRV_ShanEn  \
0    1.986813   56.665806  73.257589  73.291478  ...    0.674433    6.312695   

   HRV_FuzzyEn  HRV_MSEn  HRV_CMSEn  HRV_RCMSEn    HRV_CD   HRV_HFD   HRV_KFD  \
0     0.560191  0.477764   1.046909    0.937914  0.387085  1.933341  2.138975   

    HRV_LZC  
0  0.931561  

[1 rows x 91 columns]
